In [1]:
import numpy as np
from scipy import stats
import pandas as pd
import time

In [2]:
def calc_scores_fast(R,x,n):
    sums = np.array([None for _ in range(R)])
    sums[0] = sum([np.sign(1-x[i]) for i in range(n)])
    curr = 0
    eqs = 0
    while x[curr] == 1:
        # current index should be last idx of x where y == x[idx]
        curr += 1
        eqs += 1

    for y in range(2,R+1):
        sums[y-1] = sums[y-2] + eqs
        eqs = 0
        while curr < n and x[curr] == y:
            sums[y-1] += 1
            eqs += 1
            curr += 1
    return -np.abs(sums)

def calc_scores(R,x,n):
    return [-abs(sum([np.sign(y-x[i]) for i in range(n)])) for y in range(1,R+1)]

def round_in(x,R,n):
    x = np.sort(np.round(x))
    for i in range(n):
        if x[i] < 1:
            x[i] = 1
        elif x[i] > R:
            x[i] = R
    return x

def A(x, R, eps, Delt):
    n = len(x)
    x = round_in(x,R,n) # sort x's and round to be in {1,...,R}
    scores = calc_scores_fast(R,x,n) # calculate scores for all y vals
    
    # Report Noisy Max:
    return 1 + np.argmax([scores[i] + np.random.exponential(2*Delt/eps) for i in range(R)])

# Exponential Mechanism:
#     normalizer = sum([np.exp(eps*scores[y]/(2*Delt)) for y in range(R)])
#     prbs = [np.exp(eps*scores[y]/(2*Delt))/normalizer for y in range(R)]
#     EM = stats.rv_discrete(name='EM', values=(list(range(1,R+1)), prbs))
#     return EM.rvs()

def rank(y,x):
    n = len(x)
    x = sorted(x)
    lt = sum([1 for i in range(len(x)) if y > x[i]])
    eq = sum([1 for i in range(len(x)) if y == x[i]])
    if lt >= n/2:
        return lt
    else:
        return min(lt + eq, n/2)


In [3]:
# Universal Params
eps = .1
Delt = 1
ns = [50,100,500,2000,10000]
num_datasets = 50
runs = 10

In [4]:
# Gaussian
Gauss_Rs = [100,1000,10000]
Gauss_avg_err = pd.DataFrame(None,index=ns,columns=Gauss_Rs)
Gauss_err_std = pd.DataFrame(None,index=ns,columns=Gauss_Rs)
Gauss_err_dataset_stds = pd.DataFrame(None,index=ns,columns=Gauss_Rs)

for R in Gauss_Rs:
    for n in ns:
        start = time.time()
        dist_errs = np.array([])
        dist_dataset_avgs = np.array([])
        for j in range(num_datasets):
            dataset_errs = np.array([None for _ in range(runs)])
            x = np.random.normal(R/4,np.sqrt(R**2/10),size=(n,)) 
            for i in range(runs):
                y = A(x,R,eps,Delt)
                err = abs(n/2-rank(y,x))
                dataset_errs[i] = err
            dataset_avg = np.mean(dataset_errs)
            dist_dataset_avgs = np.append(dist_dataset_avgs,dataset_avg)
            dist_errs = np.append(dist_errs,dataset_errs)

        Gauss_avg_err.loc[n,R] = np.mean(dist_errs)
        Gauss_err_std.loc[n,R] = np.std(dist_errs)
        Gauss_err_dataset_stds.loc[n,R] = np.std(dist_dataset_avgs)

In [5]:
# rows are indexed by n values, columns indexed by R values
Gauss_avg_err.head()

,100,1000,10000
50,9.084,9.3,9.178
100,9.872,9.814,9.136
500,10.166,10.384,10.248
2000,9.662,9.002,9.71
10000,31.954,9.222,10.538


In [6]:
# rows are indexed by n values, columns indexed by R values
Gauss_err_std.head()

,100,1000,10000
50,7.233598,7.294518,7.100022
100,9.125767,9.293406,8.816434
500,9.937728,9.926759,10.184817
2000,8.783607,8.713782,10.142283
10000,19.214887,8.853514,10.356281


In [7]:
# rows are indexed by n values, columns indexed by R values
Gauss_err_dataset_stds.head()

,100,1000,10000
50,2.415232,2.531798,2.406183
100,2.972476,3.421404,2.554898
500,2.690547,3.209695,3.355845
2000,4.799995,2.831783,3.744449
10000,19.138257,3.304136,3.764088


In [8]:
# Poisson
Poi_Rs = [100,1000,10000]
Poi_avg_err = pd.DataFrame(None,index=ns,columns=Poi_Rs)
Poi_err_std = pd.DataFrame(None,index=ns,columns=Poi_Rs)
Poi_err_dataset_stds = pd.DataFrame(None,index=ns,columns=Poi_Rs)

for R in Poi_Rs:
    for n in ns:
        dist_dataset_avgs = np.array([])
        dist_errs = np.array([])
        for j in range(num_datasets):
            dataset_errs = np.array([None for _ in range(runs)])
            x = np.random.poisson(50,size=(n,)) 
            for i in range(runs):
                y = A(x,R,eps,Delt)
                err = abs(n/2-rank(y,x))
                dataset_errs[i] = err              
            dist_errs = np.append(dist_errs,dataset_errs)
            dataset_avg = np.mean(dataset_errs)
            dist_dataset_avgs = np.append(dist_dataset_avgs,dataset_avg)
        Poi_avg_err.loc[n,R] = np.mean(dist_errs)
        Poi_err_std.loc[n,R] = np.std(dist_errs)
        Poi_err_dataset_stds.loc[n,R] = np.std(dist_errs)

In [9]:
# rows are indexed by n values, columns indexed by R values
Poi_avg_err.head()

,100,1000,10000
50,14.946,23.238,24.81
100,13.376,33.852,47.536
500,1.838,1.984,1.6
2000,0.17,0.018,0.138
10000,0.0,0.0,0.0


In [10]:
# rows are indexed by n values, columns indexed by R values
Poi_err_dataset_stds.head()

,100,1000,10000
50,10.153772,5.503213,1.987436
100,16.872659,21.444675,10.224515
500,6.383397,6.238569,5.388135
2000,1.516938,0.40209,1.248582
10000,0.0,0.0,0.0


In [11]:
# rows are indexed by n values, columns indexed by R values
Poi_err_std.head()

,100,1000,10000
50,10.153772,5.503213,1.987436
100,16.872659,21.444675,10.224515
500,6.383397,6.238569,5.388135
2000,1.516938,0.40209,1.248582
10000,0.0,0.0,0.0


In [12]:
# Bimodal
R = 1000
Bi_ks = [10,100,200]
Bi_avg_err = pd.DataFrame(None,index=ns,columns=Bi_ks)
Bi_err_std = pd.DataFrame(None,index=ns,columns=Bi_ks)
Bi_err_dataset_stds = pd.DataFrame(None,index=ns,columns=Bi_ks)


for k in Bi_ks:
    for n in ns:
        bimodal = stats.rv_discrete(name='bimodal', values=([R/2-k,R/2+k], [1/2,1/2]))
        dist_errs = np.array([])
        dist_dataset_avgs = np.array([])
        for j in range(num_datasets):
            dataset_errs = np.array([None for _ in range(runs)])
            x = bimodal.rvs(size=(n,)) 
            for i in range(runs):
                y = A(x,R,eps,Delt)
                err = abs(n/2-rank(y,x))
                dataset_errs[i] = err              
            dist_errs = np.append(dist_errs,dataset_errs)
            dataset_avg = np.mean(dataset_errs)
            dist_dataset_avgs = np.append(dist_dataset_avgs,dataset_avg)
        Bi_avg_err.loc[n,k] = np.mean(dist_errs)
        Bi_err_std.loc[n,k] = np.std(dist_errs)
        Bi_err_dataset_stds.loc[n,k] = np.std(dist_errs)

In [13]:
# rows are indexed by n values, columns indexed by k values
Bi_avg_err.head()

,10,100,200
50,21.064,9.348,5.884
100,19.2,6.396,4.708
500,9.78,9.26,8.26
2000,17.88,17.42,18.78
10000,36.22,36.68,37.74


In [14]:
# rows are indexed by n values, columns indexed by k values
Bi_err_std.head()

,10,100,200
50,8.607433,10.302567,7.669846
100,22.223771,9.790566,6.074762
500,6.503199,7.042187,5.164533
2000,13.989482,13.147,14.655088
10000,29.87125,30.767151,26.213592


In [15]:
# rows are indexed by n values, columns indexed by k values
Bi_err_dataset_stds.head()

,10,100,200
50,8.607433,10.302567,7.669846
100,22.223771,9.790566,6.074762
500,6.503199,7.042187,5.164533
2000,13.989482,13.147,14.655088
10000,29.87125,30.767151,26.213592
